Segmenting and Clustering Neighborhoods in Toronto - Assignment

In [128]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -y lxml
import lxml.html as lh

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [129]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# get page contents
page = requests.get(url)

# Store contents of the webpage
doc = lh.fromstring(page.content)

# row elements in html are stored between <tr>..</tr>
row_elements = doc.xpath('//tr')


In [130]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list name
for t in row_elements[0]:
    i+=1
    name=t.text_content()
    #print(name)
    col.append((name.rstrip(),[]))

In [131]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(row_elements)):
    #T is our j'th row
    T=row_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [148]:
# Organize the data into a pandas data frame
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df = df.replace('\n','', regex=True)

# drop cells with a borough that is Not assigned
df = df[df.Borough != "Not assigned"].reset_index(drop=True)

# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

# remove last text row - not relevant
df.drop(df.tail(1).index,inplace=True)

# assign completed frame to toronto_frame
toronto_frame=df
toronto_frame.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [149]:
# print the number of rows of your dataframe
toronto_frame.shape

(103, 3)